# 🦀 RustLab Log Scale Plotting Examples

This notebook demonstrates comprehensive log scale plotting capabilities with rustlab's modern API.

**Features showcased:**
- `vec64!` and `array64!` macros for concise data creation
- Functional programming with map, filter, zip operations
- Vector concatenation operations with `concat` method
- Cross-platform Jupyter support (Windows, Linux, macOS)
- All log scale combinations (linear-log, log-linear, log-log)

In [ ]:
// Import required libraries with only needed features for faster compilation
:dep rustlab-rs = { path = "/home/poisr/rustlab-rs", features = ["plotting"] }
use rustlab::prelude::*;
use std::f64::consts::PI;

println!("🦀 RustLab Log Scale Plotting Examples");
println!("=====================================\n");

// Test cross-platform Jupyter support
println!("🖥️  Platform: {}", std::env::consts::OS);
println!("📔 Jupyter environment detected: Cross-platform support enabled");
println!("💡 Plots will display inline in Jupyter notebooks on Windows, Linux, and macOS!\n");

## 📊 1. Exponential Growth (Log-Linear Scale)

Demonstrates how exponential growth appears as a straight line on a log-linear plot.

In [ ]:
println!("📊 1. EXPONENTIAL GROWTH (Log-Linear Scale)");
println!("--------------------------------------------");

// Using vec64! macro for concise vector creation
let time = vec64![0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0];

// Functional programming: map operation for exponential growth
let population = time.map(|t| (t * 0.5).exp());

// Ergonomic slice access with to_slice() method!
println!("Time points: {:?}", time.to_slice());
println!("Population (exponential): {:?}", 
         population.to_slice().iter()
             .map(|&x| format!("{:.2}", x))
             .collect::<Vec<_>>());

In [ ]:
// Create plot with linear x-axis, log y-axis - cross-platform Jupyter support!
Plot::new()
    .line_with(&time, &population, "Exponential Growth")
    .xlabel("Time (years)")
    .ylabel("Population")
    .title("Population Growth (Linear-Log)")
    .xscale(Scale::Linear)
    .yscale(Scale::Log10)  // Log scale reveals exponential as straight line
    .show().expect("Failed to display plot");

println!("✓ Linear-Log plot displayed");

## 📈 2. Power Law Relationship (Log-Log Scale)

Power laws appear as straight lines on log-log plots, making them easy to identify and analyze.

In [ ]:
println!("📈 2. POWER LAW RELATIONSHIP (Log-Log Scale)");

// Create frequency data using linspace for realistic power law
let frequencies = linspace(1.0, 1000.0, 20);

// Create simple noise vector using vec64! macro
let noise_data = vec64![0.05, -0.02, 0.03, -0.01, 0.04, -0.03, 0.02, -0.01, 0.03, -0.02,
                       0.01, -0.03, 0.02, -0.01, 0.04, -0.02, 0.03, -0.01, 0.02, -0.03];

// Power law: y = x^(-2) + noise (Functional approach with proper type inference)
let base_amplitudes = frequencies.map(|f| f.powf(-2.0));  // Power law relationship
let amplitudes = base_amplitudes.zip_with(&noise_data, |amp, n| (amp + n).max(1e-6))
    .expect("Failed to zip amplitude data with noise");

// Use direct access without storing slice references
println!("Frequencies (first 5): {:?}", {
    let slice = frequencies.to_slice();
    slice.get(..5).unwrap_or(&[])
});
println!("Amplitudes (first 5): {:?}", {
    let slice = amplitudes.to_slice();
    slice.get(..5).unwrap_or(&[]).iter()
        .map(|&x| format!("{:.6}", x))
        .collect::<Vec<_>>()
});

In [ ]:
// Log-log plot reveals power law as straight line
Plot::new()
    .scatter_with(&frequencies, &amplitudes, "Power Law")
    .xlabel("Frequency (Hz)")
    .ylabel("Amplitude")
    .title("Power Law Relationship (Log-Log)")
    .xscale(Scale::Log10)
    .yscale(Scale::Log10)
    .show().expect("Failed to display plot");

println!("✓ Log-Log plot displayed");

## 🔬 3. Scientific Data Analysis (Multiple Experiments)

Shows how to combine multiple experimental datasets using rustlab's concatenation methods.

In [ ]:
println!("🔬 3. SCIENTIFIC DATA ANALYSIS (Multiple Experiments)");
println!("------------------------------------------------------");

// Simulate multiple experimental runs
let experiment1_time = linspace(0.1, 10.0, 50);
let experiment2_time = linspace(10.1, 20.0, 50);
let experiment3_time = linspace(20.1, 30.0, 50);

// Different decay rates for each experiment
let decay1 = experiment1_time.map(|t| 100.0 * (-t * 0.2).exp());
let decay2 = experiment2_time.map(|t| 80.0 * (-(t - 10.0) * 0.15).exp());
let decay3 = experiment3_time.map(|t| 60.0 * (-(t - 20.0) * 0.1).exp());

// Concatenate using the ergonomic vconcat! macro
let all_time_vec = vconcat![experiment1_time, experiment2_time, experiment3_time]
    .expect("Failed to concatenate time vectors");
let all_measurements_vec = vconcat![decay1, decay2, decay3]
    .expect("Failed to concatenate measurement vectors");

println!("Total data points: {}", all_time_vec.len());
if let (Some(min_time), Some(max_time)) = (all_time_vec.min(), all_time_vec.max()) {
    println!("Time range: {:.1} to {:.1}", min_time, max_time);
}

In [ ]:
// Filter out measurements below detection limit using functional approach
let detection_limit = 1.0;

let filtered_data: Vec<(f64, f64)> = {
    let time_slice = all_time_vec.to_slice();
    let meas_slice = all_measurements_vec.to_slice();
    
    time_slice.iter()
        .zip(meas_slice.iter())
        .filter(|&(_, measurement)| *measurement > detection_limit)  // Fix: use &(_, measurement) pattern
        .map(|(&t, &m)| (t, m))
        .collect()
};

// Use unzip for cleaner vector creation
let (time_data, measurement_data): (Vec<f64>, Vec<f64>) = filtered_data.into_iter().unzip();
let filtered_time = VectorF64::from(time_data);
let filtered_measurements = VectorF64::from(measurement_data);

println!("Points above detection limit: {}/{}", 
         filtered_time.len(), all_time_vec.len());

// Log-linear plot for exponential decay
Plot::new()
    .scatter_with(&filtered_time, &filtered_measurements, "Filtered Data")
    .line_with(&all_time_vec, &all_measurements_vec, "All Data")
    .xlabel("Time (s)")
    .ylabel("Signal Intensity")
    .title("Multi-Experiment Decay Analysis (Log-Linear)")
    .xscale(Scale::Linear)
    .yscale(Scale::Log10)
    .show().expect("Failed to display plot");

println!("✓ Multi-experiment analysis displayed");

## 💰 4. Financial Data Analysis (Portfolio Performance)

Demonstrates matrix operations and portfolio analysis using the `array64!` macro.

In [ ]:
println!("💰 4. FINANCIAL DATA ANALYSIS (Portfolio Performance)");
println!("------------------------------------------------------");

// Create portfolio data using array64! macro
let portfolio_returns = array64![
    [0.05, 0.12, -0.03,  0.08],   // Stock A returns over 4 periods
    [0.03, 0.08, -0.01,  0.06],   // Stock B returns
    [0.07, 0.15, -0.05,  0.10]    // Stock C returns
];

println!("Portfolio returns matrix (3 stocks × 4 periods):");
for i in 0..portfolio_returns.shape().0 {
    let mut row_data = Vec::new();
    for j in 0..portfolio_returns.shape().1 {
        row_data.push(portfolio_returns.get(i, j).unwrap());
    }
    println!("Stock {}: {:?}", 
             ('A' as u8 + i as u8) as char,
             row_data.iter()
                 .map(|&x| format!("{:+.2}", x))
                 .collect::<Vec<_>>());
}

In [ ]:
// Calculate cumulative returns (functional approach)
let initial_value = 1000.0;
let periods = vec64![1.0, 2.0, 3.0, 4.0];

// Calculate portfolio value over time for each stock
let mut cumulative_values = Vec::new();
for i in 0..portfolio_returns.shape().0 {
    let mut returns_data = Vec::new();
    for j in 0..portfolio_returns.shape().1 {
        if let Some(value) = portfolio_returns.get(i, j) {
            returns_data.push(value);
        }
    }
    
    let mut value = initial_value;
    let mut values = vec![value];
    
    for &ret in &returns_data {
        value *= (1.0 + ret);
        values.push(value);
    }
    cumulative_values.push(VectorF64::from_slice(&values[1..])); // Skip initial value
}

println!("\nCumulative portfolio values:");
for (i, values) in cumulative_values.iter().enumerate() {
    let last_value = values.to_slice().last().unwrap_or(&0.0);
    println!("Stock {}: ${:.2} → ${:.2}", 
             ('A' as u8 + i as u8) as char,
             initial_value,
             last_value);
}

In [ ]:
// Plot on linear scale to show absolute growth
Plot::new()
    .line_with(&periods, &cumulative_values[0], "Stock A")
    .line_with(&periods, &cumulative_values[1], "Stock B")
    .line_with(&periods, &cumulative_values[2], "Stock C")
    .xlabel("Period")
    .ylabel("Portfolio Value ($)")
    .title("Portfolio Performance Comparison")
    .xscale(Scale::Linear)
    .yscale(Scale::Linear)
    .show().expect("Failed to display plot");

println!("✓ Financial analysis displayed");

## 📡 5. Signal Processing (Frequency Domain Analysis)

Advanced signal processing example showing functional composition and frequency analysis.

In [ ]:
println!("📡 5. SIGNAL PROCESSING (Frequency Domain Analysis)");
println!("---------------------------------------------------");

// Generate complex signal using advanced rustlab features
let t = linspace(0.0, 2.0, 1000);

// Multi-component signal using functional composition
let signal = t.map(|time| {
    let fundamental = (2.0 * PI * 10.0 * time).sin();      // 10 Hz
    let harmonic2 = 0.5 * (2.0 * PI * 20.0 * time).sin();  // 20 Hz
    let harmonic3 = 0.25 * (2.0 * PI * 30.0 * time).sin(); // 30 Hz
    let noise = 0.1 * (time * 1000.0).sin(); // High freq noise
    fundamental + harmonic2 + harmonic3 + noise
});

if let Some(max_time) = t.max() {
    println!("Signal: {} samples over {:.1}s", t.len(), max_time);
}

In [ ]:
// Simulate FFT magnitudes (simplified for demonstration)
let freqs = linspace(1.0, 100.0, 50);
let magnitudes = freqs.map(|f| {
    // Simulate peaks at harmonics with 1/f noise floor
    let peak_10hz = if (f - 10.0).abs() < 2.0 { 10.0 } else { 0.0 };
    let peak_20hz = if (f - 20.0).abs() < 2.0 { 5.0 } else { 0.0 };
    let peak_30hz = if (f - 30.0).abs() < 2.0 { 2.5 } else { 0.0 };
    let noise_floor = 1.0 / f; // 1/f noise
    (peak_10hz + peak_20hz + peak_30hz + noise_floor).max(1e-3)
});

if let Some(max_freq) = freqs.max() {
    println!("Frequency analysis: {} points up to {:.1} Hz", 
            freqs.len(), max_freq);
}

// Find peaks using logical filtering - fix reference pattern
let threshold = 1.0;
let peak_data: Vec<(f64, f64)> = {
    let freq_slice = freqs.to_slice();
    let mag_slice = magnitudes.to_slice();
    
    freq_slice.iter()
        .zip(mag_slice.iter())
        .filter(|&(_, mag)| *mag > threshold)  // Fix: use &(_, mag) pattern
        .map(|(&f, &m)| (f, m))
        .collect()
};

// Use unzip for cleaner vector creation
let (peak_freq_data, peak_mag_data): (Vec<f64>, Vec<f64>) = peak_data.into_iter().unzip();
let peak_freqs = VectorF64::from(peak_freq_data);
let peak_mags = VectorF64::from(peak_mag_data);

// Display without storing slice reference
println!("Detected peaks at frequencies: {:?} Hz", {
    let slice = peak_freqs.to_slice();
    slice.iter().map(|&x| format!("{:.1}", x)).collect::<Vec<_>>()
});

In [ ]:
// Log-linear plot emphasizes dynamic range
Plot::new()
    .line_with(&freqs, &magnitudes, "FFT Magnitude")
    .scatter_with(&peak_freqs, &peak_mags, "Detected Peaks")
    .xlabel("Frequency (Hz)")
    .ylabel("Magnitude")
    .title("Frequency Domain Analysis (Log Scale)")
    .xscale(Scale::Linear)
    .yscale(Scale::Log10)
    .show().expect("Failed to display plot");

println!("✓ Signal processing analysis displayed");

## 📊 6. Data Quality Analysis (Statistical Summary)

Outlier detection and statistical analysis using rustlab's built-in functions.

In [ ]:
println!("📊 6. DATA QUALITY ANALYSIS (Statistical Summary)");
println!("--------------------------------------------------");

// Generate realistic measurement data 
let measurements_data = vec64![8.2, 11.5, 9.8, 12.1, 10.3, 7.9, 13.2, 9.1, 11.8, 10.7,
                              8.5, 12.3, 9.4, 11.0, 10.8, 7.6, 13.5, 8.9, 12.7, 9.3];

// Add some outliers using concatenation
let outliers = vec64![100.0, 150.0, 200.0];
let all_data = measurements_data.concat(&[&outliers]).expect("Failed to concatenate data");

// Statistical analysis using rustlab's built-in functions
println!("Dataset statistics:");
println!("  Total points: {}", all_data.len());
println!("  Mean: {:.2}", all_data.sum_elements() / all_data.len() as f64);

// Get min/max once and use them
if let (Some(min_val), Some(max_val)) = (all_data.min(), all_data.max()) {
    println!("  Min: {:.2}", min_val);
    println!("  Max: {:.2}", max_val);
    println!("  Range: {:.2}", max_val - min_val);
}

In [ ]:
// Identify outliers using statistical methods
let q75 = 15.0; // Simplified percentile
let q25 = 5.0;
let iqr = q75 - q25;
let outlier_threshold = q75 + 1.5 * iqr;

// Re-create the data to ensure it's in scope (evcxr variable persistence issue)
let measurements_data = vec64![8.2, 11.5, 9.8, 12.1, 10.3, 7.9, 13.2, 9.1, 11.8, 10.7,
                              8.5, 12.3, 9.4, 11.0, 10.8, 7.6, 13.5, 8.9, 12.7, 9.3];
let outliers = vec64![100.0, 150.0, 200.0];
let all_data = measurements_data.concat(&[&outliers]).expect("Failed to concatenate data");

let (outlier_data, clean_data): (Vec<f64>, Vec<f64>) = {
    let data_slice = all_data.to_slice();
    let outliers: Vec<f64> = data_slice.iter()
        .filter(|&&x| x > outlier_threshold)
        .copied()
        .collect();
    
    let clean: Vec<f64> = data_slice.iter()
        .filter(|&&x| x <= outlier_threshold)
        .copied()
        .collect();
    
    (outliers, clean)
};

println!("\nOutlier analysis:");
println!("  Threshold: {:.2}", outlier_threshold);
println!("  Outliers found: {}", outlier_data.len());
println!("  Clean data points: {}", clean_data.len());

// Create histogram-like data for visualization
let indices = vec64![1.0, 2.0, 3.0, 4.0, 5.0]; // Simplified bins
let clean_counts = vec64![25.0, 45.0, 20.0, 8.0, 2.0]; // Simulated histogram
let outlier_counts = vec64![0.0, 0.0, 0.0, 0.0, 3.0]; // Outliers in highest bin

In [ ]:
// Log scale emphasizes the outlier detection
Plot::new()
    .line_with(&indices, &clean_counts, "Normal Data")
    .scatter_with(&indices, &outlier_counts, "Outliers")
    .xlabel("Data Range (bins)")
    .ylabel("Count")
    .title("Data Quality Analysis (Log Scale)")
    .xscale(Scale::Linear)
    .yscale(Scale::Log10)
    .show().expect("Failed to display plot");

println!("✓ Data quality analysis displayed");

## 🎯 Summary

This notebook demonstrated all the key features of rustlab's log scale plotting capabilities:

In [ ]:
println!("🎯 RUSTLAB IDIOMS DEMONSTRATED:");
println!("===============================");
println!("✓ vec64! and array64! macros for concise creation");
println!("✓ Functional programming: map, filter, zip_with operations");
println!("✓ Vector concatenation with chained append method");
println!("✓ Logical filtering with filter and collect");
println!("✓ Ergonomic slice access with to_slice() method");
println!("✓ Array operations (shape access, element access)");
println!("✓ Advanced data transformations");
println!("✓ Log scale visualization for all data types\n");

println!("🚀 LOG SCALE BENEFITS:");
println!("======================");
println!("• Exponential data appears linear (linear-log)");
println!("• Power laws appear linear (log-log)");
println!("• Wide dynamic range visualization");
println!("• Outlier detection and emphasis");
println!("• Scientific data standard practice");

println!("\n🎉 All log scale examples completed successfully!");
println!("RustLab provides powerful, idiomatic tools for scientific computing!");